In [1]:
# Импорт необходимых библиотек
import requests                           # Для выполнения HTTP-запросов к API
import datetime                           # Для работы с датами и временем
from datetime import timedelta
!pip install pulp
import pulp                               # Для решения задачи MILP (смешанное целочисленное линейное программирование)
import numpy as np                        # Для числовых операций
# Для модели градиентного бустинга можно использовать, например, xgboost,
# но здесь мы создадим имитацию (заглушку) предсказания.
import xgboost as xgb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 55.9 MB/s eta 0:00:00


In [2]:
# Класс для представления сегмента (соединения) между станциями
class Connection:
    def __init__(self, start, end, departure, arrival, cost, mode):
        self.start = start              # Название или код станции отправления
        self.end = end                  # Название или код станции прибытия
        self.departure = departure      # Время отправления (datetime)
        self.arrival = arrival          # Время прибытия (datetime)
        self.cost = cost                # Стоимость билета
        self.mode = mode                # Тип транспорта (например, "Поезд", "Автобус", "Самолет")

    def __repr__(self):
        return f"{self.mode} из {self.start} в {self.departure.strftime('%H:%M')} до {self.end} в {self.arrival.strftime('%H:%M')} (Стоимость: {self.cost})"

In [13]:

def fetch_yandex_schedule(from_station, to_station, date):
    """
    Получает данные расписания с API Яндекс. Путешествия.
    Для демонстрации используется заглушка – в реальном коде здесь необходимо
    выполнить запрос к API, передав параметры: from, to, date, apikey и др.
    """
    # Здесь нужно сформировать параметры запроса согласно документации API
    params = {
        "apikey": "bc8c6b0c-b611-4dfa-b587-f4d399f36050",
        "format": "json",
        "from": from_station,
        "to": to_station,
        "lang": "ru",
        "date": date.strftime("%Y-%m-%d")
    }
    url = 'https://api.rasp.yandex.net/v3.0/schedule/'

    # Выполняем запрос
    try:
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print("Ошибка при получении данных от Yandex API:", response.status_code)
            return []
        data = response.json()
    except Exception as e:
        print("Ошибка при выполнении запроса:", e)
        data = {}

    connections = []
    for segment in data.get("segments", []):
        try:
            departure = datetime.datetime.fromisoformat(segment.get("departure"))
            arrival = datetime.datetime.fromisoformat(segment.get("arrival"))
            cost = segment.get("price", 0)
            mode = segment.get("transport_type", "Unknown")
            start = segment.get("from", {}).get("title", "Unknown")
            end = segment.get("to", {}).get("title", "Unknown")
            conn = Connection(start, end, departure, arrival, cost, mode)
            connections.append(conn)
        except Exception as e:
            print("Ошибка обработки сегмента:", e)
    # Если нет реальных данных, создадим примерные соединения для демонстрации:
    if not connections:
        print("Используются демонстрационные данные (заглушка).")
        # Пример данных – два сегмента, образующих маршрут от A до D через B
        connections = [
            Connection("A", "B", datetime.datetime(date.year, date.month, date.day, 8, 0),
                       datetime.datetime(date.year, date.month, date.day, 9, 30), 50, "Поезд"),
            Connection("B", "D", datetime.datetime(date.year, date.month, date.day, 10, 30),
                       datetime.datetime(date.year, date.month, date.day, 12, 0), 80, "Автобус")
        ]
    return connections


In [6]:
# Заглушка для получения данных о погоде с Яндекс. Погода
def fetch_weather(station):
    """
    Возвращает имитацию погодных условий для заданной станции.
    В реальном решении здесь необходимо делать запрос к API Яндекс. Погода.
    """
    # Пример: для станции с плохой погодой можно вернуть "снег", иначе "ясно"
    weather_conditions = {"A": "ясно", "B": "облачно", "D": "дождь"}
    return weather_conditions.get(station, "ясно")

# Заглушка для получения данных о дорожной ситуации с Яндекс. Карты
def fetch_traffic(station):
    """
    Возвращает имитацию дорожной ситуации (фактор пробок) для заданной точки.
    В реальном решении здесь необходимо делать запрос к API Яндекс. Карты.
    """
    # Пример: значение 1.0 – нормальная ситуация, >1.0 – усиленные пробки
    traffic_conditions = {"A": 1.0, "B": 1.2, "D": 1.5}
    return traffic_conditions.get(station, 1.0)

In [7]:
# Функция для предсказания времени пересадки (в минутах) на основе данных о погоде и пробках
def predict_transfer_time(station):
    """
    Предсказывает время, необходимое для пересадки в заданном пункте.
    Здесь можно использовать обученную модель градиентного бустинга.
    В демонстрационных целях возвращаем значение, зависящее от погодных и дорожных условий.
    """
    weather = fetch_weather(station)
    traffic = fetch_traffic(station)
    # Базовое время пересадки, например, 15 минут
    base_time = 15
    # Если погода плохая или пробки сильные, увеличиваем время
    if weather in ["дождь", "снег"]:
        base_time += 10
    if traffic > 1.2:
        base_time += 5
    return base_time  # в минутах

In [8]:
# Функция для предсказания стоимости такси (с помощью модели градиентного бустинга)
def predict_taxi_cost(station):
    """
    Предсказывает стоимость такси для перемещения между различными пересадочными точками.
    Здесь можно использовать обученную модель градиентного бустинга.
    Для демонстрации возвращается значение, зависящее от погодных условий и пробок.
    """
    weather = fetch_weather(station)
    traffic = fetch_traffic(station)
    base_cost = 300  # базовая стоимость такси, например, 300 руб.
    if weather in ["дождь", "снег"]:
        base_cost *= 1.3
    if traffic > 1.2:
        base_cost *= 1.2
    return base_cost  # в рублях

In [9]:
# Функция для оценки вероятности задержки рейса (для авиарейсов)
def predict_delay_probability(connection):
    """
    Предсказывает вероятность задержки рейса.
    Для самолетов учитываем погодные условия, для других видов транспорта вероятность задержки невысока.
    """
    if connection.mode.lower() == "самолет":
        weather = fetch_weather(connection.start)
        if weather in ["дождь", "снег"]:
            return 0.5  # 50% вероятность задержки
        else:
            return 0.1  # 10% вероятность задержки
    else:
        return 0.05  # для поездов и автобусов

In [10]:
# ФУНКЦИИ ДЛЯ ОПТИМИЗАЦИИ MILP
# ------------------------------------------------------------------------------

def build_milp_model(connections, user_start, user_end, weight_time=1.0, weight_cost=1.0):
    """
    Строит MILP модель для выбора оптимального маршрута с учетом пересадок.
    Используются бинарные переменные для каждого соединения.

    Аргументы:
        connections: список объектов Connection (кандидаты для маршрута)
        user_start: станция отправления (строка)
        user_end: станция назначения (строка)
        weight_time: вес для минимизации времени (в минутах)
        weight_cost: вес для минимизации стоимости (билет + пересадка)
    Возвращает:
        MILP модель (объект pulp.LpProblem) и словарь переменных x.
    """
    # Приводим времена отправления и прибытия к числовому виду (минуты с начала дня)
    for conn in connections:
        conn.departure_minutes = conn.departure.hour * 60 + conn.departure.minute
        conn.arrival_minutes = conn.arrival.hour * 60 + conn.arrival.minute

    # Определяем MILP задачу: минимизация целевой функции
    prob = pulp.LpProblem("Route_Optimization", pulp.LpMinimize)

    # Создаем бинарные переменные для каждого соединения
    x = {}
    for i, conn in enumerate(connections):
        x[i] = pulp.LpVariable(f"x_{i}", cat="Binary")

    # Целевая функция: сумма (время в пути + стоимость билетов)
    # Здесь время рассчитывается как разница между временем прибытия и отправления для каждого сегмента
    prob += pulp.lpSum([
        x[i] * (weight_time * (conn.arrival_minutes - conn.departure_minutes) + weight_cost * conn.cost)
        for i, conn in enumerate(connections)
    ]), "Total_Travel_Time_and_Cost"

    # Список всех станций (уникальные названия)
    stations = set()
    for conn in connections:
        stations.add(conn.start)
        stations.add(conn.end)

    # Добавляем потоковые ограничения (консервация потока)
    for station in stations:
        # Если станция является отправной точкой
        if station == user_start:
            prob += pulp.lpSum([x[i] for i, conn in enumerate(connections) if conn.start == station]) == 1, f"Flow_start_{station}"
        # Если станция является конечной точкой
        elif station == user_end:
            prob += pulp.lpSum([x[i] for i, conn in enumerate(connections) if conn.end == station]) == 1, f"Flow_end_{station}"
        else:
            # Для промежуточных станций: входящие соединения = исходящие соединения
            prob += (pulp.lpSum([x[i] for i, conn in enumerate(connections) if conn.end == station]) -
                     pulp.lpSum([x[i] for i, conn in enumerate(connections) if conn.start == station])) == 0, f"Flow_{station}"

    # Добавляем ограничения по времени пересадки для последовательных соединений
    # Для каждой пары соединений (i, j), где конец i совпадает с началом j, требуем:
    # (j.departure_minutes - i.arrival_minutes) >= (предсказанное время пересадки + 30) при условии, что обе переменные равны 1.
    M = 10000  # большое число для Big-M метода
    for i, conn_i in enumerate(connections):
        for j, conn_j in enumerate(connections):
            if conn_i.end == conn_j.start:
                # Вычисляем предсказанное время пересадки для станции (в минутах)
                transfer_time = predict_transfer_time(conn_i.end)
                # Ограничение: если обе переменные выбраны (x_i = x_j = 1), то:
                # (departure_j - arrival_i) >= transfer_time + 30
                prob += (conn_j.departure_minutes - conn_i.arrival_minutes) >= (transfer_time + 30) - M * (2 - x[i] - x[j]), f"Transfer_{i}_{j}"

    return prob, x


In [11]:
def extract_route(connections, x_vars):
    """
    Извлекает выбранный маршрут (последовательность соединений) из решения MILP.
    Предполагается, что выбранные переменные образуют связную цепочку.
    """
    selected = [conn for i, conn in enumerate(connections) if pulp.value(x_vars[i]) > 0.5]
    # Сортируем выбранные соединения по времени отправления
    selected.sort(key=lambda c: c.departure)
    return selected

In [ ]:
# ОСНОВНОЙ БЛОК РАБОТЫ СКРИПТА
# ------------------------------------------------------------------------------

if __name__ == "__main__":
    # Входные параметры пользователя
    # Замените идентификаторы на реальные коды станций согласно API Яндекс.
    user_from_station = "s9600396"  # Идентификатор отправной станции
    user_to_station = "s9600396"    # Идентификатор станции назначения
    travel_date = datetime.datetime(2025, 4, 20)  # Дата поездки
    # Критерий ранжирования: можно задать веса для времени и стоимости
    weight_time = 1.0
    weight_cost = 1.0

    print("Получение данных расписания с Яндекс API...")
    connections = fetch_yandex_schedule(user_from_station, user_to_station, travel_date)
    if not connections:
        print("Нет данных о соединениях.")
        exit(1)
    else:
        print("Полученные соединения:")
        for conn in connections:
            print("  ", conn)

    # Для MILP-модели необходимо знать реальные названия начальной и конечной станций.
    # В данном примере используем названия из первого соединения.
    user_start = connections[0].start
    user_end = connections[-1].end

    # Строим MILP модель для оптимизации маршрута
    print("\nПостроение MILP модели для оптимального маршрута...")
    prob, x_vars = build_milp_model(connections, user_start, user_end, weight_time, weight_cost)

    # Решаем задачу MILP с помощью встроенного решателя pulp
    prob.solve()
    print("Статус решения:", pulp.LpStatus[prob.status])

    # Извлекаем оптимальный маршрут
    route = extract_route(connections, x_vars)
    if not route:
        print("Оптимальный маршрут не найден.")
    else:
        print("\nОптимальный маршрут:")
        for seg in route:
            print("  ", seg)

        # Дополнительные рекомендации и расчёты для пересадок
        if len(route) > 1:
            print("\nРасчёт дополнительных параметров для пересадок:")
            for i in range(len(route) - 1):
                # Предсказанное время пересадки
                transfer_station = route[i].end
                predicted_transfer = predict_transfer_time(transfer_station)
                # Предсказанная стоимость такси для пересадки
                taxi_cost = predict_taxi_cost(transfer_station)
                print(f"На пересадке в {transfer_station}:")
                print(f"  Предполагаемое время пересадки: {predicted_transfer} мин.")
                print(f"  Предполагаемая стоимость такси: {taxi_cost:.2f} руб.")

        # Оценка вероятности задержки для каждого сегмента (например, для самолетов)
        print("\nОценка вероятности задержки:")
        for seg in route:
            delay_prob = predict_delay_probability(seg)
            print(f"  {seg.mode} из {seg.start} в {seg.end}: вероятность задержки {delay_prob*100:.1f}%")

        # Вывод общей информации о маршруте
        total_time = sum([(seg.arrival - seg.departure).total_seconds() for seg in route]) / 60  # в минутах
        total_cost = sum([seg.cost for seg in route])
        print("\nОбщие характеристики маршрута:")
        print(f"  Общее время в пути (без учета пересадок): {total_time:.1f} мин.")
        print(f"  Общая стоимость билетов: {total_cost:.2f} руб.")

        # Рекомендации могут быть дополнительно скорректированы на основе истории предпочтений клиентов,
        # используя, например, методы кластеризации (K-means/K-modes/K-prototypes) для выделения оптимальных вариантов.
        print("\nРекомендуется учитывать указанные дополнительные расходы и вероятность задержек при выборе маршрута.")